# Capstone Project: Data Collection

### Imports

In [1]:
import pandas as pd
import requests 
import json
from bs4 import BeautifulSoup

### Datasets

In order to model a store-locator recommender, a dataset needs to be generated with the following data points:
- City/Town-level demographics data
- Retail venues per city
- Venue category (i.e. Coffee Shop, Parks, etc.)
- Geographic data
- User generated data

#### Cities and Towns in the San Francisco Bay Area

The first thing to collect is a list of city and towns in the San Francisco Bay Area. Using requests and BeautifulSoup to parse through Wikipedia, a dataframe was generated with the information stated above. 

In [2]:
bay_area_wiki = "https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_the_San_Francisco_Bay_Area"

In [3]:
wiki_req = requests.get(bay_area_wiki)

In [4]:
wiki_soup = BeautifulSoup(wiki_req.content,"lxml")

In [5]:
# finds table
table = wiki_soup.find("table",{"class":"wikitable plainrowheaders sortable"})
wiki_table = []

# for loop created to collect data from wikipedia table, to be stored in a dictionary and appended to a list
for row in table.find("tbody").find_all("tr"):
    cities = {}
    
    try:
        cities["area"] = row.find_all("a")[0].text
        cities["county"] = row.find_all("a")[1].text
        cities["city_or_town"] = row.find("td").text.replace("\n","").split()[0]
    except:
        pass
    
    wiki_table.append(cities)
    
# dataframe generated from list of dictionaries is saved onto variable for future use
wiki_df = pd.DataFrame(wiki_table[2:])    


In [6]:
wiki_df.head()

,area,county,city_or_town
0,Alameda,Alameda,City
1,Albany,Alameda,City
2,American Canyon,Napa,City
3,Antioch,Contra Costa,City
4,Atherton,San Mateo,Town


#### Geopoints of California Cities

Most geolocation data (Latitude and Longitude) are sourced from the public OpenDataSoft dataset on US Zip Code Latitude and Longitude and combined with the wikipedia dataframe above. Nulls are manually filled with data sourced from Google. 

In [7]:
# Source: https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/information/?refine.state=CA

ca_geopoints = pd.read_csv("../data/us-zip-code-latitude-and-longitude.csv",
                          sep = ";")

In [8]:
ca_geopoints.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,91131,Pasadena,CA,33.786594,-118.298662,-8,1,"33.786594,-118.298662"
1,92399,Yucaipa,CA,34.033564,-117.037760,-8,1,"34.033564,-117.03776"
2,95957,Meridian,CA,39.070323,-121.830650,-8,1,"39.070323,-121.83065"
3,93225,Frazier Park,CA,34.815266,-119.004040,-8,1,"34.815266,-119.00404"
4,94244,Sacramento,CA,38.377411,-121.444429,-8,1,"38.377411,-121.444429"


In [9]:
ca_geopoints.drop(columns = ["State",
                             "Timezone",
                             "Daylight savings time flag",
                             "Zip"], 
                 inplace = True)

In [10]:
combined_df = pd.merge(wiki_df,
                       ca_geopoints,
                       right_on = "City",
                       left_on = "area",
                       how = "left"
                      )

In [11]:
combined_df.drop(columns = "City", inplace = True)

In [12]:
combined_df.isnull().sum()

area            0
county          0
city_or_town    0
Latitude        9
Longitude       9
geopoint        9
dtype: int64

In [13]:
# Source: Google

to_replace = {"Belvedere": [37.8727,-122.4644,"37.8727,-122.4644"],
              "Colma": [37.6749,-122.4562,"37.6749,-122.4562"],
              "East Palo Alto": [37.4688,-122.1411,"37.4688,-122.1411"],
              "Hillsborough": [37.5741,-122.3794,"37.5741,-122.3794"],
              "Los Altos Hills": [37.3797,-122.1375,"37.3797,-122.1375"],
              "Monte Sereno": [37.2363,-121.9925,"37.2363,-121.9925"],
              "St. Helena": [38.511241,-122.45695,"38.511241,-122.45695"],
              "Tiburon": [37.8735,-122.4566,"37.8735,-122.4566"],
              "Woodside": [37.4299,-122.2539,"37.4299,-122.2539"],
             }

for city in list(combined_df[combined_df["Latitude"].isna()]["area"]):
    combined_df.loc[combined_df[combined_df["area"] == city].index, ["Latitude","Longitude","geopoint"]] = to_replace[city]

In [14]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413 entries, 0 to 412
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area          413 non-null    object 
 1   county        413 non-null    object 
 2   city_or_town  413 non-null    object 
 3   Latitude      413 non-null    float64
 4   Longitude     413 non-null    float64
 5   geopoint      413 non-null    object 
dtypes: float64(2), object(4)
memory usage: 38.8+ KB


#### Population and Income Data

Demographics data (population, median household income and income per capita) are collected through parsing the US government's census website using both requests and BeautifulSoup. Due to the nature of the site, if a city is not found, information obtained is on the whole of the US. To mitigate this issue, numbers are replaced with those found on bestplaces.net. 

In [15]:
combined_df["population"] = 0
combined_df["median_household_income"] = 0
combined_df["income_per_capita"] = 0

census_url = "https://www.census.gov/quickfacts/fact/table/"

# for loop is created to be able to get demographics data from the US census website
for i in combined_df.index:
    area = combined_df.loc[i,"area"].lower()
    area_type = combined_df.loc[i,"city_or_town"].lower()
    state = "california"
    combined_url = census_url + area + area_type + state
    rem_spaces = combined_url.replace(" ","")
    req = requests.get(rem_spaces)
    soup = BeautifulSoup(req.content,"lxml")
    ls_pop = []
    ls_income =[]
    for row in soup.find("tbody",{"data-topic":"Population"}).find_all("tr"):
        for value in row.find_all("td"):   
            ls_pop.append(value.text.replace("\n","").replace(",",""))
    # if-else statement is created so that if the latest data (April 2020) is not available to use 2019 estimates instead
    if ls_pop[-1] != "X": 
        combined_df.loc[i, "population"] = int(ls_pop[-1])
    else:
        combined_df.loc[i, "population"] = int(ls_pop[1])
    for row in soup.find("tbody",{"data-topic":"Income & Poverty"}).find_all("td"):
        ls_income.append(row.text.replace("\n","").replace("$","").replace(",","").replace("+",""))
    combined_df.loc[i, "median_household_income"] = int(ls_income[1])
    combined_df.loc[i, "income_per_capita"] = int(ls_income[3]) 
    print(area)

alameda
alameda
albany
american canyon
antioch
antioch
atherton
belmont
belmont
belvedere
benicia
berkeley
berkeley
berkeley
berkeley
berkeley
berkeley
berkeley
berkeley
berkeley
berkeley
berkeley
brentwood
brisbane
burlingame
burlingame
burlingame
calistoga
campbell
campbell
campbell
clayton
cloverdale
colma
concord
concord
concord
concord
concord
concord
concord
concord
corte madera
corte madera
cotati
cotati
cupertino
cupertino
daly city
daly city
daly city
daly city
danville
danville
dixon
dublin
east palo alto
el cerrito
emeryville
emeryville
fairfax
fairfax
fairfield
fairfield
foster city
fremont
fremont
fremont
fremont
fremont
gilroy
gilroy
half moon bay
hayward
hayward
hayward
hayward
hayward
hayward
hayward
healdsburg
hercules
hillsborough
lafayette
larkspur
larkspur
livermore
livermore
los altos
los altos
los altos
los altos hills
los gatos
los gatos
los gatos
los gatos
martinez
menlo park
menlo park
menlo park
mill valley
mill valley
millbrae
millbrae
milpitas
milpitas
monte

In [16]:
combined_df.head()

,area,county,city_or_town,Latitude,Longitude,geopoint,population,median_household_income,income_per_capita
0,Alameda,Alameda,City,37.734379,-122.239520,"37.734379,-122.23952",77624,104756,52448
1,Alameda,Alameda,City,37.769528,-122.259370,"37.769528,-122.25937",77624,104756,52448
2,Albany,Alameda,City,37.889125,-122.293710,"37.889125,-122.29371",19696,95400,50089
3,American Canyon,Napa,City,38.170372,-122.256050,"38.170372,-122.25605",20475,101792,36148
4,Antioch,Contra Costa,City,37.920852,-121.723919,"37.920852,-121.723919",111502,76601,29591


In [17]:
# first city seemed to be duplicated, and therefore dropped

combined_df.drop_duplicates(inplace = True)

In [18]:
combined_df.describe()

,Latitude,Longitude,population,median_household_income,income_per_capita
count,300.000000,300.000000,3.000000e+02,300.000000,300.000000
mean,37.736063,-122.210858,9.114676e+06,111489.016667,57064.290000
std,0.363605,0.303455,5.344354e+07,36027.266735,22346.556801
min,36.986148,-122.997000,5.247000e+03,53198.000000,20769.000000
25%,37.396677,-122.412015,4.317000e+04,85530.000000,43133.750000
50%,37.757865,-122.243605,1.053550e+05,109593.000000,50619.000000
75%,37.952140,-121.957183,4.330310e+05,125626.500000,68883.000000
max,38.795668,-121.497349,3.314493e+08,250000.000000,165947.000000


In [19]:
combined_df[combined_df["population"] == combined_df["population"].max()]

,area,county,city_or_town,Latitude,Longitude,geopoint,population,median_household_income,income_per_capita
9,Belvedere,Marin,City,37.872700,-122.464400,"37.8727,-122.4644",331449281,62843,34103
23,Brisbane,San Mateo,City,37.682882,-122.404220,"37.682882,-122.40422",331449281,62843,34103
33,Colma,San Mateo,Town,37.674900,-122.456200,"37.6749,-122.4562",331449281,62843,34103
106,Monte Sereno,Santa Clara,City,37.236300,-121.992500,"37.2363,-121.9925",331449281,62843,34103
182,Portola Valley,San Mateo,Town,37.374421,-122.213000,"37.374421,-122.213",331449281,62843,34103
199,Ross,Marin,Town,38.022322,-122.561689,"38.022322,-122.561689",331449281,62843,34103
200,St. Helena,Napa,City,38.511241,-122.456950,"38.511241,-122.45695",331449281,62843,34103
412,Yountville,Napa,Town,38.403011,-122.362280,"38.403011,-122.36228",331449281,62843,34103


In [20]:
# Source: https://www.bestplaces.net
to_update = {"Belvedere": [2156,250001,166281],
             "Brisbane": [4642,119833,57652],
             "Colma": [1410,104531,39871],
             "Monte Sereno": [3551,250001,129053],
             "Portola Valley": [4589,250001,140002],
             "Ross": [2311,228438,109871],
             "St. Helena": [6056,95909,54350],
             "Yountville": [2978,115536,55832],
             }

for city in list(combined_df[combined_df["population"] == combined_df["population"].max()]["area"]):
    combined_df.loc[combined_df[combined_df["area"] == city].index, ["population","median_household_income","income_per_capita"]] = to_update[city]

In [61]:
combined_df.to_csv("../data/combined_df.csv", index = False)

#### Foursquare Places API

Using Foursquare Places API, data on top recommended venues per city/town in the San Francisco Bay Area is obtained. The following information are collected:
- Name of Venue
- Address of Venue
- Latitude and Longitude Data of Venue
- Venue Catefory

The collected dataframe is then merged with the above dataframe and imported as a csv file.

*Note: Foursquare API has a limit of 50 venues per requests.*

In [22]:
def foursquare_places(df):
    
    '''Function takes in dataframe (with geopoint and area name data) and returns a dataframe with the name,
    address, city, geopoint (individual lat, long and combined), and venue category per inputted geopoints 
    (max. 50 venues per geopoint) using Foursquare Places API'''
    
    ls = []
    base_url = 'https://api.foursquare.com/v2/venues/explore'
    for geopoint in df["geopoint"]:
        params = {
            "client_id":"FOURSQUARE_CLIENT_ID",
            "client_secret":"FOURSQUARE_CLIENT_SECRET",
            "ll":geopoint,
            "limit":"50",
            "v":"20210527"}
        req = requests.get(url=base_url, params = params)
        try:
            assert(req.status_code == 200)
            data = json.loads(req.text)["response"]
            for venues in data["groups"][0]["items"]:
                table = venues["venue"]
                dic = {}
                dic["name"] = table["name"]
                dic["address"] = table["location"]["formattedAddress"][0]
                dic["city"] = df[df["geopoint"] == geopoint]["area"].values[0]
                dic["geopoint"] = geopoint
                dic["lat"] = table["location"]["lat"]
                dic["long"] = table["location"]["lng"]
                dic["categories"] = table["categories"][0]["name"]
                if dic not in ls:
                    ls.append(dic)
                else:
                    pass
            print(f'Location collected: {len(ls)}')
            time.sleep(20)
        except:
            print(f'Error Status Code: {req.status_code}')
    return pd.DataFrame(ls)

In [23]:
foursquare_df = foursquare_places(combined_df)

Location collected: 50
Error Status Code: 200
Location collected: 100
Error Status Code: 200
Location collected: 150
Error Status Code: 200
Location collected: 200
Error Status Code: 200
Location collected: 250
Error Status Code: 200
Location collected: 300
Error Status Code: 200
Location collected: 350
Error Status Code: 200
Location collected: 400
Error Status Code: 200
Location collected: 450
Error Status Code: 200
Location collected: 500
Error Status Code: 200
Location collected: 550
Error Status Code: 200
Location collected: 600
Error Status Code: 200
Location collected: 650
Error Status Code: 200
Location collected: 700
Error Status Code: 200
Location collected: 750
Error Status Code: 200
Location collected: 800
Error Status Code: 200
Location collected: 850
Error Status Code: 200
Location collected: 900
Error Status Code: 200
Location collected: 950
Error Status Code: 200
Location collected: 1000
Error Status Code: 200
Location collected: 1050
Error Status Code: 200
Location col

Location collected: 7339
Error Status Code: 200
Location collected: 7339
Error Status Code: 200
Location collected: 7339
Error Status Code: 200
Location collected: 7389
Error Status Code: 200
Location collected: 7389
Error Status Code: 200
Location collected: 7439
Error Status Code: 200
Location collected: 7489
Error Status Code: 200
Location collected: 7539
Error Status Code: 200
Location collected: 7589
Error Status Code: 200
Location collected: 7639
Error Status Code: 200
Location collected: 7689
Error Status Code: 200
Location collected: 7739
Error Status Code: 200
Location collected: 7789
Error Status Code: 200
Location collected: 7839
Error Status Code: 200
Location collected: 7889
Error Status Code: 200
Location collected: 7939
Error Status Code: 200
Location collected: 7989
Error Status Code: 200
Location collected: 8039
Error Status Code: 200
Location collected: 8089
Error Status Code: 200
Location collected: 8139
Error Status Code: 200
Location collected: 8189
Error Status Co

In [24]:
foursquare_df.isnull().sum()

name          0
address       0
city          0
geopoint      0
lat           0
long          0
categories    0
dtype: int64

In [25]:
foursquare_df.head()

,name,address,city,geopoint,lat,long,categories
0,Chuck Corica Golf Complex,1 Clubhouse Memorial Rd,Alameda,"37.734379,-122.23952",37.738885,-122.234326,Golf Course
1,Shoreline Park,"Alameda, CA",Alameda,"37.734379,-122.23952",37.742580,-122.250126,Park
2,Angel Fish Japanese Cuisine,883 C-2 Island Dr,Alameda,"37.734379,-122.23952",37.737478,-122.240107,Japanese Restaurant
3,La Penca Azul,891 Island Dr Ste B (at Mecartney Rd),Alameda,"37.734379,-122.23952",37.737667,-122.240767,Mexican Restaurant
4,La Val's Pizza,891 Island Dr,Alameda,"37.734379,-122.23952",37.737610,-122.241001,Italian Restaurant


In [26]:
foursquare_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13039 entries, 0 to 13038
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        13039 non-null  object 
 1   address     13039 non-null  object 
 2   city        13039 non-null  object 
 3   geopoint    13039 non-null  object 
 4   lat         13039 non-null  float64
 5   long        13039 non-null  float64
 6   categories  13039 non-null  object 
dtypes: float64(2), object(5)
memory usage: 713.2+ KB


In [36]:
merged_df = pd.merge(combined_df,
                    foursquare_df,
                    left_on = "geopoint",
                    right_on = "geopoint",
                    how = "left"
                   )

In [37]:
merged_df.head()

,area,county,city_or_town,Latitude,Longitude,geopoint,population,median_household_income,income_per_capita,name,address,city,lat,long,categories
0,Alameda,Alameda,City,37.734379,-122.23952,"37.734379,-122.23952",77624,104756,52448,Chuck Corica Golf Complex,1 Clubhouse Memorial Rd,Alameda,37.738885,-122.234326,Golf Course
1,Alameda,Alameda,City,37.734379,-122.23952,"37.734379,-122.23952",77624,104756,52448,Shoreline Park,"Alameda, CA",Alameda,37.742580,-122.250126,Park
2,Alameda,Alameda,City,37.734379,-122.23952,"37.734379,-122.23952",77624,104756,52448,Angel Fish Japanese Cuisine,883 C-2 Island Dr,Alameda,37.737478,-122.240107,Japanese Restaurant
3,Alameda,Alameda,City,37.734379,-122.23952,"37.734379,-122.23952",77624,104756,52448,La Penca Azul,891 Island Dr Ste B (at Mecartney Rd),Alameda,37.737667,-122.240767,Mexican Restaurant
4,Alameda,Alameda,City,37.734379,-122.23952,"37.734379,-122.23952",77624,104756,52448,La Val's Pizza,891 Island Dr,Alameda,37.737610,-122.241001,Italian Restaurant


In [60]:
merged_df.to_csv("../data/location_full.csv", index = False)

In [39]:
merged_df.shape

(14939, 15)

#### Google Places API

Using Google Places API, aggregate venue data on ratings, number of users who rated and price level are collected. Because two different APIs are used, some venues did not have "atmosphere" data. To mitigate the issue for enterprise venues, a branch with no data will be given the average value for the enterprise branches collected, whilst unique venues with no ratings is filled with 0. The dataframe is then combined, duplicate columns are dropped (city and geopoints which includes the same information as area and Lat/Long columns), and saved to a csv for EDA and modeling purposes. 

In [32]:
def google_places(df):
    
    '''Function takes in dataframe (with geopoint and area name data) and returns a dataframe with 
    venue name, geopoint, price_level, rating and numbers of users who rated per venue using Google Places API'''
    
    ls = []
    base_url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json'
    for venue in df["name"].unique():
        area = df[df["name"] == venue]["city"].values[0]
        geopoints = df[df["name"] == venue]["geopoint"].values[0]
        params = {"input": venue + " "+ area,
                  "inputtype":"textquery",
                  "fields":"price_level,rating,user_ratings_total",
                  "key":"GOOGLE_API_KEY"}
        req = requests.get(url=base_url, params = params)
        try:
            assert(req.status_code == 200)
            data = json.loads(req.text)["candidates"][0]
            dic = {}
            dic["name"] = venue
            dic["geopoint"] = geopoints
            dic["price_level"] = data["price_level"]
            dic["rating"] = data["rating"]
            dic["user_ratings_total"] = data["user_ratings_total"]
            if dic not in ls:
                ls.append(dic)
            else:
                pass
            print(f'Location collected: {len(ls)}')
            time.sleep(20)
        except:
            print(f'Error Status Code: {req.status_code}')
    return pd.DataFrame(ls)

In [33]:
google_df = google_places(foursquare_df)

Error Status Code: 200
Error Status Code: 200
Location collected: 1
Error Status Code: 200
Location collected: 2
Error Status Code: 200
Location collected: 3
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 4
Error Status Code: 200
Error Status Code: 200
Location collected: 5
Error Status Code: 200
Error Status Code: 200
Location collected: 6
Error Status Code: 200
Error Status Code: 200
Location collected: 7
Error Status Code: 200
Location collected: 8
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 9
Error Status Code: 200
Location collected: 10
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 11
Error Status Code: 200
Location collected: 12
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 13
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 14
Error Status Code: 2

Location collected: 132
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 133
Error Status Code: 200
Error Status Code: 200
Location collected: 134
Error Status Code: 200
Location collected: 135
Error Status Code: 200
Location collected: 136
Error Status Code: 200
Location collected: 137
Error Status Code: 200
Location collected: 138
Error Status Code: 200
Location collected: 139
Error Status Code: 200
Error Status Code: 200
Location collected: 140
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 141
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 142
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 143
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 144
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 1

Location collected: 243
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 244
Error Status Code: 200
Error Status Code: 200
Location collected: 245
Error Status Code: 200
Location collected: 246
Error Status Code: 200
Location collected: 247
Error Status Code: 200
Location collected: 248
Error Status Code: 200
Location collected: 249
Error Status Code: 200
Location collected: 250
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 251
Error Status Code: 200
Location collected: 252
Error Status Code: 200
Location collected: 253
Error Status Code: 200
Location collected: 254
Error Status Code: 200
Location collected: 255
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 256
Error Status Code: 200
Location collected: 257
Error Status Code: 200
Error Status Code: 200
Location collected: 258
Error Status Code: 200
Error Status Code:

Location collected: 376
Error Status Code: 200
Location collected: 377
Error Status Code: 200
Location collected: 378
Error Status Code: 200
Error Status Code: 200
Location collected: 379
Error Status Code: 200
Error Status Code: 200
Location collected: 380
Error Status Code: 200
Error Status Code: 200
Location collected: 381
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 382
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 383
Error Status Code: 200
Error Status Code: 200
Location collected: 384
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 385
Error Status Code: 200
Location collected: 386
Error Status Code: 200
Error Status Code: 200
Location collected: 387
Error Status Code: 200
Location collected: 388
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 389
Error Status Code: 200
Location collected: 

Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 494
Error Status Code: 200
Location collected: 495
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 496
Error Status Code: 200
Error Status Code: 200
Location collected: 497
Error Status Code: 200
Location collected: 498
Error Status Code: 200
Error Status Code: 200
Location collected: 499
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 500
Error Status Code: 200
Location collected: 501
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 502
Error Status Code: 200
Location collected: 503
Error Status Code: 200
Location collected: 504
Error Status Code: 200
Location collected: 505
Error Status Code: 200
Error Status Code: 200

...

Location collected: 2854
Error Status Code: 200
Location collected: 2855
Error Status Code: 200
Error Status Code: 200
Location collected: 2856
Error Status Code: 200
Location collected: 2857
Error Status Code: 200
Location collected: 2858
Error Status Code: 200
Location collected: 2859
Error Status Code: 200
Location collected: 2860
Error Status Code: 200
Error Status Code: 200
Location collected: 2861
Error Status Code: 200
Error Status Code: 200
Location collected: 2862
Error Status Code: 200
Error Status Code: 200
Location collected: 2863
Error Status Code: 200
Error Status Code: 200
Location collected: 2864
Error Status Code: 200
Location collected: 2865
Error Status Code: 200
Error Status Code: 200
Location collected: 2866
Error Status Code: 200
Location collected: 2867
Error Status Code: 200
Location collected: 2868
Error Status Code: 200
Location collected: 2869
Error Status Code: 200
Location collected: 2870
Error Status Code: 200
Location collected: 2871
Error Status Code: 20

Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 2989
Error Status Code: 200
Location collected: 2990
Error Status Code: 200
Error Status Code: 200
Location collected: 2991
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 2992
Error Status Code: 200
Error Status Code: 200
Location collected: 2993
Error Status Code: 200
Location collected: 2994
Error Status Code: 200
Location collected: 2995
Error Status Code: 200
Location collected: 2996
Error Status Code: 200
Location collected: 2997
Error Status Code: 200
Location collected: 2998
Error Status Code: 200
Location collected: 2999
Error Status Code: 200
Location collected: 3000
Error Status Code: 200
Location collected: 3001
Error Status Code: 200
Error Status Code: 200
Location collected: 3002
Error Status Code: 200
Location collected: 3003
Error Status Code: 200
Location collected: 3004
Error Status Code: 200
Error Status Code: 200
Lo

Location collected: 3113
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 3114
Error Status Code: 200
Location collected: 3115
Error Status Code: 200
Error Status Code: 200
Location collected: 3116
Error Status Code: 200
Error Status Code: 200
Location collected: 3117
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 3118
Error Status Code: 200
Location collected: 3119
Error Status Code: 200
Location collected: 3120
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 3121
Error Status Code: 200
Location collected: 3122
Error Status Code: 200
Location collected: 3123
Error Status Code: 200
Location collected: 3124
Error Status Code: 200
Location collected: 3125
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location

Error Status Code: 200
Location collected: 3224
Error Status Code: 200
Location collected: 3225
Error Status Code: 200
Location collected: 3226
Error Status Code: 200
Error Status Code: 200
Location collected: 3227
Error Status Code: 200
Location collected: 3228
Error Status Code: 200
Error Status Code: 200
Location collected: 3229
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 3230
Error Status Code: 200
Location collected: 3231
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 3232
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Location collected: 3233
Error Status Code: 200
Error Status Code: 200
Location collected: 3234
Error Status Code: 200
Error Status Code: 200
Location collected: 3235
Error Status Code: 200
Error Status Code: 200
Location collected: 3236
Error Status Code: 200
Location collected: 3237
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Locati

Error Status Code: 200
Location collected: 3343
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200
Error Status Code: 200


In [34]:
google_df.head()

,name,geopoint,price_level,rating,user_ratings_total
0,Angel Fish Japanese Cuisine,"37.734379,-122.23952",2,4.3,112
1,La Penca Azul,"37.734379,-122.23952",2,4.3,2263
2,La Val's Pizza,"37.734379,-122.23952",2,4.4,190
3,Trader Joe's,"37.734379,-122.23952",2,4.6,1038
4,Trabocco Kitchen and Cocktails,"37.734379,-122.23952",2,4.6,852


In [40]:
rated_df = pd.merge(merged_df,
                    google_df,
                    left_on = ["name","geopoint"],
                    right_on = ["name","geopoint"],
                    how = "left"
                   )

In [41]:
rated_df.drop(columns = ["city","geopoint"], inplace = True)
rated_df.rename(columns = {"Latitude": "lat_area",
                 "Longitude": "long_area",
                 "lat": "lat_venue",
                 "long": "long_venue"
                }, inplace = True)

In [42]:
rated_df.shape

(14939, 16)

In [43]:
rated_df.isnull().sum()

area                           0
county                         0
city_or_town                   0
lat_area                       0
long_area                      0
population                     0
median_household_income        0
income_per_capita              0
name                           0
address                        0
lat_venue                      0
long_venue                     0
categories                     0
price_level                11068
rating                     11068
user_ratings_total         11068
dtype: int64

In [47]:
unique_google = google_df.groupby("name").mean()

In [55]:
for i in rated_df[rated_df["price_level"].isna()].index:
    for name in unique_google.index.unique():
        if rated_df.loc[i,"name"] == name:
            rated_df.loc[i, ["price_level","rating","user_ratings_total"]] = list(unique_google.loc[name, ["price_level","rating","user_ratings_total"]])

In [56]:
rated_df.isnull().sum()

area                          0
county                        0
city_or_town                  0
lat_area                      0
long_area                     0
population                    0
median_household_income       0
income_per_capita             0
name                          0
address                       0
lat_venue                     0
long_venue                    0
categories                    0
price_level                6574
rating                     6574
user_ratings_total         6574
dtype: int64

In [57]:
rated_df.fillna(0, inplace = True)

In [59]:
rated_df.to_csv("../data/location_with_google_ratings.csv", index = False)

*Note: Due to the taxing nature of collecting the data above, multiple dataframes are saved onto a csv throughout the process for potential future use (so the code does not have to be constantly re-run)*